In [61]:
import pandas as pd
def cut_and_paste_column(input_path: str, output_path: str):
    try:
        print(f"[**] Reading Excel file: '{input_path}'")
        df = pd.read_excel(input_path, engine='openpyxl')

        if df.shape[1] < 38:
            print("[ERROR] The input file does not have enough columns (needs at least 38).")
            return

        # Cut the 38th column (AL)
        print("[*] Cutting the 38th column (AL)...")
        al_column_data = df.pop(df.columns[37])

        r_index = 17

        print("[*] Pasting the cut column between Q and R...")
        df.insert(r_index, 'AL', al_column_data)

        # Save the modified DataFrame to a new Excel file
        print(f"[*] Saving the modified file to: '{output_path}'")
        df.to_excel(output_path, index=False, engine='openpyxl')
        print("[SUCCESS] File saved successfully.")
    
    except FileNotFoundError: # will only happen is the file path is not found.
        print(f"[ERROR] The file was not found: {input_path}")
    except Exception as e: # will only happen when the file is opened somewhere.
        print(f"[ERROR] An error occurred:(file is opened somewhere) {e}")
if __name__ == "__main__":
    # Replace these paths with your actual file paths
    input_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Master WLP L0 Super Collector as on 23-SEP-2025.xlsx"
    output_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    
    cut_and_paste_column(input_path=input_excel_file, output_path=output_excel_file)

[**] Reading Excel file: 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Master WLP L0 Super Collector as on 23-SEP-2025.xlsx'
[*] Cutting the 38th column (AL)...
[*] Pasting the cut column between Q and R...
[*] Saving the modified file to: 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'


C:\Users\mangoel\AppData\Local\Temp\ipykernel_28336\3619529699.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(r_index, 'AL', al_column_data)


[SUCCESS] File saved successfully.


In [62]:
import pandas as pd
import sys
from collections import defaultdict

def _mangle_duplicate_columns(columns):
    counts = defaultdict(int)
    new_columns = []
    for col in columns:
        if not col or str(col).isspace():
            col = 'Unnamed'
        if counts[col] > 0:
            new_columns.append(f"{col}_{counts[col]}")
        else:
            new_columns.append(col)
        counts[col] += 1
    return new_columns

def transform_client_details_robust_v2(input_path: str, output_path: str):
    try:
        print(f"[*] Reading Excel file without a default header: '{input_path}'")
        full_df = pd.read_excel(input_path, header=None, usecols=range(1, 40), engine='openpyxl')

    except FileNotFoundError:
        print(f"[ERROR] The file was not found: {input_path}")
        sys.exit(1)
    except Exception as e:
        print(f"[ERROR] An error occurred while reading the Excel file: {e}")
        sys.exit(1)

    super_headers = full_df.iloc[3].ffill().tolist() 
    original_headers = full_df.iloc[4].tolist() 
    
    print("[*] Original headers found:", original_headers)
    main_headers = _mangle_duplicate_columns(original_headers)
    print("[*] Headers after de-duplication:", main_headers)

    df = full_df.iloc[5:].copy()
    df.columns = main_headers
    df.reset_index(drop=True, inplace=True)

    print(f"[*] Successfully loaded {df.shape[0]} rows and {df.shape[1]} columns.")
    
    static_cols = main_headers[:17] 
    base_repeated_names = _mangle_duplicate_columns(original_headers[16:21])
    
    processed_dfs = []

    for i in range(4):
        start_col_index = 17 + (i * 5)
        end_col_index = start_col_index + 5
        
        source_block_name = super_headers[start_col_index]
        print(f"[*] Processing Block {i+1} (Source: '{source_block_name}')...")
        
        current_block_cols = main_headers[start_col_index:end_col_index]
        temp_df = df[static_cols + current_block_cols].copy()
        
        rename_mapping = dict(zip(current_block_cols, base_repeated_names))
        temp_df.rename(columns=rename_mapping, inplace=True)
        processed_dfs.append(temp_df)

    print("[*] Stacking the processed blocks...")
    final_df = pd.concat(processed_dfs, ignore_index=True)
    final_df.dropna(subset=base_repeated_names, how='all', inplace=True)
    
    print(f"[*] Transformation complete. New file will have {final_df.shape[0]} rows.")
    try:
        final_df.to_excel(output_path, index=False, engine='openpyxl')
        print(f"[SUCCESS] The transformed file has been saved as '{output_path}'")
    except Exception as e:
        print(f"[ERROR] Could not save the file: {e}")

# --- How to use the script ---
if __name__ == "__main__":
    # Please ensure these paths are correct on your system
    input_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    # Using a new output file name to avoid confusion
    output_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    
    transform_client_details_robust_v2(input_path=input_excel_file, output_path=output_excel_file)

[*] Reading Excel file without a default header: 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'
[*] Original headers found: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'CLIENTDETAILS-1', nan, nan, nan, nan, 'CLIENTDETAILS-2', nan, nan, nan, nan, 'CLIENTDETAILS-3', nan, nan, nan, nan, 'CLIENTDETAILS-4', nan, nan, nan, nan, nan, nan]
[*] Headers after de-duplication: [nan, 'nan_1', 'nan_2', 'nan_3', 'nan_4', 'nan_5', 'nan_6', 'nan_7', 'nan_8', 'nan_9', 'nan_10', 'nan_11', 'nan_12', 'nan_13', 'nan_14', 'nan_15', 'nan_16', 'CLIENTDETAILS-1', 'nan_17', 'nan_18', 'nan_19', 'nan_20', 'CLIENTDETAILS-2', 'nan_21', 'nan_22', 'nan_23', 'nan_24', 'CLIENTDETAILS-3', 'nan_25', 'nan_26', 'nan_27', 'nan_28', 'CLIENTDETAILS-4', 'nan_29', 'nan_30', 'nan_31', 'nan_32', 'nan_33', 'nan_34']
[*] Successfully loaded 2654 rows and 39 columns.
[*] Processing Block 1 (Source: 'Source Site')...
[*] Processing Block 2 (Source: 'Sour

In [63]:
import pandas as pd

def remove_header_row(input_path: str, output_path: str):
    """
    Reads an Excel file, removes the header row (r=0), and saves the modified DataFrame to a new Excel file.
    """
    try:
        # Load the Excel file without treating the first row as the header
        print(f"[*] Reading Excel file: '{input_path}'")
        df = pd.read_excel(input_path, header=None, engine='openpyxl')  # Read without headers

        # Remove the first row (header row)
        print("[*] Removing the header row (r=0)...")
        df = df.iloc[1:].reset_index(drop=True)  # Skip the first row and reset the index

        # Save the modified DataFrame to a new Excel file
        print(f"[*] Saving the modified file to: '{output_path}'")
        df.to_excel(output_path, index=False, header=False, engine='openpyxl')  # Save without headers
        print("[SUCCESS] File saved successfully.")
    
    except FileNotFoundError:
        print(f"[ERROR] The file was not found: {input_path}")
    except Exception as e:
        print(f"[ERROR] An error occurred: {e}")

# --- How to use the script ---
if __name__ == "__main__":
    # Replace these paths with your actual file paths
    input_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    output_excel_file =r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    
    remove_header_row(input_path=input_excel_file, output_path=output_excel_file)

[*] Reading Excel file: 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'
[*] Removing the header row (r=0)...
[*] Saving the modified file to: 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'
[SUCCESS] File saved successfully.


In [64]:
import pandas as pd
import os

INPUT_FILENAME = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
OUTPUT_FILENAME = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"

#    You can add more columns here, e.g., ['COMMONPORT', 'Rate', 'CardType']
COLUMNS_TO_COPY = ['OPSSH-SL']
ADDITIONAL_MATCH_COLUMN = 'AIMOTR/WL5e CLIENTPORT'
# -------------------------------

def process_snc_protection_paths(input_path: str, output_path: str, columns_to_process: list, match_column: str):

    try:
        df = pd.read_excel(input_path)
        print(f"Successfully loaded data from '{input_path}'. It has {df.shape[1]} columns.")
    except FileNotFoundError:
        print(f"Error: The file '{input_path}' was not found. Please check the INPUT_FILENAME.")
        return
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")
        return
    required_columns = ['SNCName', match_column] + columns_to_process
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        print(f"Error: The input file is missing the following required columns: {missing_cols}")
        return

    # --- 3. Process the Data ---
    # Create a copy to store our changes. This is safer than modifying the original dataframe.
    df_updated = df.copy()
    update_count = 0
    
    # Filter for rows where 'SNCName' contains '-W-'
    working_rows = df[df['SNCName'].str.contains('-W-', na=False)]
    print(f"Found {len(working_rows)} 'Working' paths to process.")

    # Iterate over each "Working" row found
    for index, w_row in working_rows.iterrows():
        source_snc = w_row['SNCName']
        source_match_value = w_row[match_column]  # Value from the additional match column
        
        # Construct the "Protection" SNCName by replacing '-W-' with '-P-'
        target_snc = source_snc.replace('-W-', '-P-', 1)

        # Find the index of the matching "Protection" row in our copy
        target_indices = df_updated.index[
            (df_updated['SNCName'] == target_snc) & 
            (df_updated[match_column] == source_match_value)
        ].tolist()

        if not target_indices:
            continue
        
        # Assuming SNCNames are unique and match_column ensures uniqueness, we take the first match
        target_index = target_indices[0]

        # Now, loop through the columns we need to copy for this matched pair
        for column_name in columns_to_process:
            value_to_copy = w_row[column_name]
            
            # Check if there is a value to copy (handles empty strings, None, NaN)
            if pd.notna(value_to_copy) and value_to_copy != '':
                # Copy the value to the target row and specified column
                df_updated.loc[target_index, column_name] = value_to_copy
                update_count += 1
            else:
                print(f"     - Skipping column '{column_name}' as source value is empty.")

    # --- 4. Save the Result ---
    # The df_updated DataFrame contains all original columns and data, plus our changes.
    try:
        df_updated.to_excel(output_path, index=False)
    except Exception as e:
        print(f"\nAn error occurred while saving the file: {e}")

# This block runs the main function when the script is executed
if __name__ == "__main__":
    
    # Execute the main processing logic with the settings from the configuration section
    process_snc_protection_paths(
        input_path=INPUT_FILENAME,
        output_path=OUTPUT_FILENAME,
        columns_to_process=COLUMNS_TO_COPY,
        match_column=ADDITIONAL_MATCH_COLUMN
    )

Successfully loaded data from 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'. It has 22 columns.
Error: The input file is missing the following required columns: ['AIMOTR/WL5e CLIENTPORT']


In [65]:
import pandas as pd

def create_combined_column(input_path: str, output_path: str, new_column_name: str):
    """
    Reads an Excel file, looks into a specific sheet, and creates a new column
    by combining values from the columns 'SOURCENE/OriginatingNode', 'OPSSH-SL', and 'OPSNO'.
    
    Args:
        input_path (str): Path to the input Excel file.
        sheet_name (str): Name of the sheet to process.
        output_path (str): Path to save the modified Excel file.
        new_column_name (str): Name of the new column to be created.
    """
    try:
        # Load the specific sheet into a DataFrame
        print(f"[*] Reading Excel file: '{input_path}'")
        df = pd.read_excel(input_path, engine='openpyxl')

        # Ensure the required columns exist
        required_columns = ["SOURCENE/OriginatingNode", "OPSSH-SL", "OPSNO"]
        for col in required_columns:
            if col not in df.columns:
                print(f"[ERROR] The column '{col}' does not exist in the sheet.")
                return

        # Create the new column by combining the specified columns without spaces
        print(f"[*] Creating new column: '{new_column_name}'...")
        df[new_column_name] = (
            df["SOURCENE/OriginatingNode"].astype(str) +
            df["OPSSH-SL"].astype(str) +
            df["OPSNO"].astype(str)
        )

        # Save the modified DataFrame to a new Excel file
        print(f"[*] Saving the modified file to: '{output_path}'...")
        df.to_excel(output_path, index=False, engine='openpyxl')
        print("[SUCCESS] File saved successfully.")
    
    except FileNotFoundError:
        print(f"[ERROR] The file was not found: {input_path}")
    except Exception as e:
        print(f"[ERROR] An error occurred: {e}")

# --- How to use the script ---
if __name__ == "__main__":
    # Replace these paths and parameters with your actual file paths and column name
    input_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    
    output_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    new_column_name = "NE-SHELFSLOT-OPSSW"  # Replace with the name of the new column

    create_combined_column(
        input_path=input_excel_file,
        output_path=output_excel_file,
        new_column_name=new_column_name
    )

[*] Reading Excel file: 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'
[*] Creating new column: 'NE-SHELFSLOT-OPSSW'...
[*] Saving the modified file to: 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'...
[SUCCESS] File saved successfully.


In [66]:
import pandas as pd

def filter_and_split_to_sheets(input_path: str, output_path: str, column_name: str, filter_value1, filter_value2):

    try:
        # Load the Excel file into a DataFrame
        print(f"[*] Reading Excel file: '{input_path}'")
        df = pd.read_excel(input_path, engine='openpyxl')

        # Ensure the specified column exists
        if column_name not in df.columns:
            print(f"[ERROR] The column '{column_name}' does not exist in the input file.")
            return

        # Filter rows based on the specified values
        print(f"[*] Filtering rows based on '{column_name}'...")
        df_filter1 = df[df[column_name] == filter_value1]  # Rows matching filter_value1
        df_filter2 = df[df[column_name] == filter_value2]  # Rows matching filter_value2

        # Write the filtered data into separate sheets in a new Excel file
        print(f"[*] Writing filtered data to '{output_path}'...")
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            df_filter1.to_excel(writer, sheet_name=f"Filter_{filter_value1}", index=False)
            df_filter2.to_excel(writer, sheet_name=f"Filter_{filter_value2}", index=False)
        print("[SUCCESS] File saved successfully.")
    
    except FileNotFoundError:
        print(f"[ERROR] The file was not found: {input_path}")
    except Exception as e:
        print(f"[ERROR] An error occurred: {e}")

# --- How to use the script ---
if __name__ == "__main__":
    # Replace these paths and parameters with your actual file paths and filter values
    input_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    output_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    column_to_filter = "CH TYPE"  # Replace with the actual column name
    filter_value_1 = "W"  # Replace with the first filter value
    filter_value_2 = "P"  # Replace with the second filter value

    filter_and_split_to_sheets(
        input_path=input_excel_file,
        output_path=output_excel_file,
        column_name=column_to_filter,
        filter_value1=filter_value_1,
        filter_value2=filter_value_2
    )

[*] Reading Excel file: 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'
[*] Filtering rows based on 'CH TYPE'...
[*] Writing filtered data to 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'...
[SUCCESS] File saved successfully.


In [67]:
import pandas as pd

def filter_column_and_save(input_path: str, output_path: str, sheet_to_filter: str, column_name: str, filter_values: list):
    """
    Filters rows based on multiple values in a specific column in a specific sheet and writes the result 
    to a single sheet while including other sheets unchanged in the output Excel file.

    Args:
        input_path (str): Path to the input Excel file.
        output_path (str): Path to the output Excel file.
        sheet_to_filter (str): Name of the sheet to apply the filter on.
        column_name (str): Name of the column to apply the filter on.
        filter_values (list): List of values to filter on.
    """
    try:
        # Load the Excel file with all sheets
        print(f"[*] Reading Excel file: '{input_path}'")
        all_sheets = pd.read_excel(input_path, sheet_name=None, engine='openpyxl')  # Load all sheets into a dictionary

        # Check if the specified sheet exists
        if sheet_to_filter not in all_sheets:
            print(f"[ERROR] The sheet '{sheet_to_filter}' does not exist in the input file.")
            return

        # Process the specific sheet
        print(f"[*] Filtering sheet: '{sheet_to_filter}'...")
        df_to_filter = all_sheets[sheet_to_filter]  # Get the specific sheet as a DataFrame

        # Ensure the specified column exists in the sheet
        if column_name not in df_to_filter.columns:
            print(f"[ERROR] The column '{column_name}' does not exist in the sheet '{sheet_to_filter}'.")
            return

        # Filter rows based on the specified values
        filtered_df = df_to_filter[df_to_filter[column_name].isin(filter_values)]  # Filter rows matching any of the values

        # Update the sheet with the filtered data
        all_sheets[sheet_to_filter] = filtered_df

        # Write all sheets to the new Excel file
        print(f"[*] Writing all sheets to '{output_path}'...")
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            for sheet_name, sheet_data in all_sheets.items():
                sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)  # Write each sheet to the output file
        print("[SUCCESS] File saved successfully.")
    
    except FileNotFoundError:
        print(f"[ERROR] The file was not found: {input_path}")
    except Exception as e:
        print(f"[ERROR] An error occurred: {e}")

# --- How to use the script ---
if __name__ == "__main__":
    # Replace these paths and parameters with your actual file paths and filter values
    input_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    output_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    sheet_to_filter = "Filter_W"  # Replace with the name of the sheet to filter
    column_to_filter = "COMMONPORT"  # Replace with the actual column name
    filter_values = ["COMMON IN/OUT(11/12)", "COMMON IN/OUT(17/18)", "COMMON IN/OUT(23/24)", "COMMON IN/OUT(5/6)"]  # Replace with the filter values

    filter_column_and_save(
        input_path=input_excel_file,
        output_path=output_excel_file,
        sheet_to_filter=sheet_to_filter,
        column_name=column_to_filter,
        filter_values=filter_values
    )


[*] Reading Excel file: 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'
[*] Filtering sheet: 'Filter_W'...
[*] Writing all sheets to 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'...
[SUCCESS] File saved successfully.


In [68]:
import pandas as pd
import os

def clean_and_copy_excel_sheets(input_excel_path: str, output_excel_path: str, sheet_to_process: str, column_to_check: str):
    """
    Reads an entire Excel workbook, filters a specific sheet to remove rows
    where a particular column is blank, and saves the cleaned sheet along
    with all other original sheets to a new Excel file.

    Args:
        input_excel_path (str): The full path to the source Excel file.
        output_excel_path (str): The full path for the cleaned output Excel file.
        sheet_to_process (str): The name of the sheet to clean.
        column_to_check (str): The name of the column to check for blank values.
    """
    print("--- Ciena Multi-Sheet Data Processing Script Initialized ---")

    # --- 1. Validate Input File ---
    if not os.path.exists(input_excel_path):
        print(f"Error: The input file was not found at '{input_excel_path}'")
        return

    try:
        # --- 2. Read ALL sheets from the Excel file into a dictionary of DataFrames ---
        # By setting sheet_name=None, pandas reads all sheets.
        # The result is a dictionary where keys are sheet names and values are the DataFrames.
        print(f"Reading all sheets from '{input_excel_path}'...")
        all_sheets_dict = pd.read_excel(input_excel_path, sheet_name=None)
        print(f"Successfully read {len(all_sheets_dict)} sheets: {list(all_sheets_dict.keys())}")

        # --- 3. Check if the sheet to be processed actually exists ---
        if sheet_to_process not in all_sheets_dict:
            print(f"Error: The sheet named '{sheet_to_process}' was not found in the Excel file.")
            print(f"Available sheets are: {list(all_sheets_dict.keys())}")
            return

        # --- 4. Get the specific DataFrame to clean ---
        df_to_clean = all_sheets_dict[sheet_to_process]

        # --- 5. Check if the specified column exists in the target sheet ---
        if column_to_check not in df_to_clean.columns:
            print(f"Error: Column '{column_to_check}' not found in sheet '{sheet_to_process}'.")
            print(f"Available columns in '{sheet_to_process}' are: {list(df_to_clean.columns)}")
            return

        # --- 6. The Core Logic: Filter and Remove Blank Rows from the target sheet ---
        print(f"\nProcessing sheet: '{sheet_to_process}'...")
        initial_rows = len(df_to_clean)
        print(f"Initial number of rows in '{sheet_to_process}': {initial_rows}")

        # The .notna() method identifies cells that are NOT blank (not NA/NaN).
        cleaned_df = df_to_clean[df_to_clean[column_to_check].notna()]

        final_rows = len(cleaned_df)
        rows_removed = initial_rows - final_rows
        print(f"Filtering complete. Removed {rows_removed} row(s) where '{column_to_check}' was blank.")
        print(f"Final number of rows in '{sheet_to_process}': {final_rows}")

        # --- 7. Update the dictionary with the cleaned DataFrame ---
        # We replace the original DataFrame for the processed sheet with our new, cleaned one.
        all_sheets_dict[sheet_to_process] = cleaned_df

        # --- 8. Save ALL sheets to the new Excel file using ExcelWriter ---
        # ExcelWriter is the tool for writing multiple sheets to one file.
        print(f"\nSaving all sheets to '{output_excel_path}'...")
        with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
            for sheet_name, df in all_sheets_dict.items():
                print(f"  - Writing sheet: '{sheet_name}'")
                # Write each DataFrame (df) to a sheet with its original name (sheet_name)
                # index=False prevents pandas from writing the row numbers as a column.
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        print("\n--- Process Completed Successfully! New file created. ---")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# ==============================================================================
# --- MAIN EXECUTION BLOCK ---
# ==============================================================================
if __name__ == "__main__":
    # --- CONFIGURE YOUR FILE AND COLUMN SETTINGS HERE ---

    # 1. Set the full path to your input Excel file.
    # Using a raw string (r'...') is helpful on Windows to avoid issues with backslashes.
    input_file = r'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'

    # 2. Set the name of the ONE sheet you want to clean.
    # This must match the sheet name in the file exactly (it is case-sensitive).
    sheet_name_to_clean = "Filter_P"

    # 3. Set the name of the column to check for blank values within that sheet.
    column_name_to_filter = "OPSNO"

    # 4. Set the full path for your new, cleaned output file.
    output_file = r'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'

    # --- DO NOT EDIT BELOW THIS LINE ---
    # Call the function with your settings
    clean_and_copy_excel_sheets(
        input_excel_path=input_file,
        output_excel_path=output_file,
        sheet_to_process=sheet_name_to_clean,
        column_to_check=column_name_to_filter
    )

--- Ciena Multi-Sheet Data Processing Script Initialized ---
Reading all sheets from 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'...
Successfully read 2 sheets: ['Filter_W', 'Filter_P']

Processing sheet: 'Filter_P'...
Initial number of rows in 'Filter_P': 4148
Filtering complete. Removed 2065 row(s) where 'OPSNO' was blank.
Final number of rows in 'Filter_P': 2083

Saving all sheets to 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'...
  - Writing sheet: 'Filter_W'
  - Writing sheet: 'Filter_P'

--- Process Completed Successfully! New file created. ---


In [69]:
import pandas as pd

def filter_excel_sheets(input_filename, output_filename):
    """
    Reads an Excel file, filters rows in each sheet based on a specific
    value in the 'Channel Project' column, and saves to a new file.

    Args:
        input_filename (str): The name of the source Excel file.
        output_filename (str): The name for the new, filtered Excel file.
    """
    try:
        # Read all sheets from the input Excel file into a dictionary of DataFrames
        # Setting sheet_name=None tells pandas to read all sheets
        all_sheets_df = pd.read_excel(input_filename, sheet_name=None, engine='openpyxl')
        print(f"Successfully loaded '{input_filename}'. Found sheets: {list(all_sheets_df.keys())}")

    except FileNotFoundError:
        print(f"Error: The file '{input_filename}' was not found. Please make sure it's in the same directory as the script.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    # Define the column to filter and the value to remove
    column_to_filter = "Channel Project"
    value_to_remove = "L0 Channel (FOTR)"

    # Create a Pandas Excel writer using openpyxl as the engine.
    # The 'with' statement ensures the writer is properly closed.
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        print("\nProcessing sheets...")
        # Loop through the dictionary of sheets (sheet_name: DataFrame)
        for sheet_name, df in all_sheets_df.items():
            print(f"  - Processing sheet: '{sheet_name}'")

            # Check if the target column exists in the current sheet's DataFrame
            if column_to_filter in df.columns:
                initial_rows = len(df)
                
                # Filter the DataFrame to keep rows where the column value is NOT the one to remove
                filtered_df = df[df[column_to_filter] != value_to_remove]
                
                final_rows = len(filtered_df)
                rows_removed = initial_rows - final_rows

                print(f"    -> Found '{column_to_filter}' column. Removed {rows_removed} row(s).")
                
                # Write the filtered DataFrame to the new Excel file, preserving the sheet name
                # index=False prevents pandas from writing the DataFrame index as a column
                filtered_df.to_excel(writer, sheet_name=sheet_name, index=False)

            else:
                # If the column doesn't exist, write the original sheet to the new file without changes
                print(f"    -> Column '{column_to_filter}' not found. Copying sheet as is.")
                df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"\nProcessing complete. Filtered data has been saved to '{output_filename}'")


if __name__ == "__main__":
    # --- Configuration ---
    # Change these file names to match your files
    input_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    output_excel_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
    
    # --- Run the function ---
    filter_excel_sheets(input_excel_file, output_excel_file)

Successfully loaded 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'. Found sheets: ['Filter_W', 'Filter_P']

Processing sheets...
  - Processing sheet: 'Filter_W'
    -> Found 'Channel Project' column. Removed 10 row(s).
  - Processing sheet: 'Filter_P'
    -> Found 'Channel Project' column. Removed 10 row(s).

Processing complete. Filtered data has been saved to 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx'


In [ ]:
# 1. Imports
import pandas as pd

# 2. Configuration
# --- File and Sheet Parameters ---
PRIMARY_FILE_PATH = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
REFERENCE_FILE_PATH = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\WLP_NMS_services_UTL_comparsion_file_31_July_2025.xlsx"

PRIMARY_SHEET_NAME = 'Filter_W'
REFERENCE_SHEET_NAME = 'Link name reversed'

# --- Column Name Parameters ---
PRIMARY_COLUMN_NAME = 'Link Name'
REFERENCE_COLUMN_NAME = 'Unnamed: 4'


# --- Helper Function for Standardization ---
def create_canonical_key(city_string):

    try:
        # Split the string by ' - ' and strip whitespace from each part
        parts = sorted([part.strip() for part in city_string.split(' - ', 1)])
        return '|'.join(parts)
    except:
        # Return a non-matching key for blank or malformed cells
        return None


# 3. Data Loading
print("Step 1: Loading data from Excel files...")

# Load the primary data file, keeping all original columns.
df_primary = pd.read_excel(
    io=PRIMARY_FILE_PATH,
    sheet_name=PRIMARY_SHEET_NAME,
)
# Store the original column order to use for the final output
original_columns = df_primary.columns.tolist()
print(f"Primary file loaded successfully with {len(original_columns)} columns.")

# Load ONLY the required column from the reference file to save memory.
df_reference = pd.read_excel(
    io=REFERENCE_FILE_PATH,
    sheet_name=REFERENCE_SHEET_NAME,
    usecols=[REFERENCE_COLUMN_NAME]
)
# Drop any empty rows from the reference column
df_reference.dropna(subset=[REFERENCE_COLUMN_NAME], inplace=True)
print("Reference file loaded.")


# 4. Transformation Logic
print("Step 2: Processing and standardizing data...")

# --- Part A: Build the Canonical Mapping Dictionary ---
# Create a highly efficient lookup map (dictionary).
# The key is the standardized format (e.g., 'CityA|CityB').
# The value is the desired format from the reference file (e.g., 'CityA - CityB').
print("Building a reference map of correct connection names...")
canonical_map = {
    create_canonical_key(name): name 
    for name in df_reference[REFERENCE_COLUMN_NAME]
}

# --- Part B: Apply the Transformation to the Primary DataFrame ---
# Create the standardized key for the primary column.
df_primary['canonical_key'] = df_primary[PRIMARY_COLUMN_NAME].apply(create_canonical_key)

# Use the map to find the correct format.
# If a key exists in our map, use the map's value; otherwise, keep the original name.
# This preserves names that don't have a match in the reference file.
df_primary[PRIMARY_COLUMN_NAME] = df_primary['canonical_key'].map(canonical_map).fillna(df_primary[PRIMARY_COLUMN_NAME])

print("Data transformation complete. All other columns are preserved.")


# 5. Data Export
print("Step 3: Preparing final output and updating the original Excel file...")

# --- Part A: Assemble the Final DataFrame ---
# Select only the original columns in their original order.
# This step drops the temporary 'canonical_key' helper column.
df_final = df_primary[original_columns]

# --- Part B: Export to Original Excel File ---
# Use ExcelWriter in append mode with 'replace' to overwrite the specific sheet
# while leaving other sheets in the workbook untouched.
try:
    with pd.ExcelWriter(
        PRIMARY_FILE_PATH,
        mode="a",
        engine="openpyxl",
        if_sheet_exists="replace"
    ) as writer:
        df_final.to_excel(writer, sheet_name=PRIMARY_SHEET_NAME, index=False)
    
    print(f"✅ Success! Sheet '{PRIMARY_SHEET_NAME}' in '{PRIMARY_FILE_PATH}' has been updated.")

except FileNotFoundError:
    print(f"❌ Error: The file was not found at {PRIMARY_FILE_PATH}. Please check the path.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

Step 1: Loading data from Excel files...
Primary file loaded successfully with 23 columns.


ValueError: Worksheet named 'Link name reversed' not found

In [ ]:
# 1. Imports
import pandas as pd

# 2. Configuration
# --- File and Sheet Parameters ---
PRIMARY_FILE_PATH = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
REFERENCE_FILE_PATH = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\WLP_NMS_services_UTL_comparsion_file_31_July_2025.xlsx"

PRIMARY_SHEET_NAME = 'Filter_P'
REFERENCE_SHEET_NAME = 'Link name reversed'

# --- Column Name Parameters ---
PRIMARY_COLUMN_NAME = 'Link Name'
REFERENCE_COLUMN_NAME = 'Unnamed: 4'


# --- Helper Function for Standardization ---
def create_canonical_key(city_string):

    try:
        # Split the string by ' - ' and strip whitespace from each part
        parts = sorted([part.strip() for part in city_string.split(' - ', 1)])
        return '|'.join(parts)
    except:
        # Return a non-matching key for blank or malformed cells
        return None


# 3. Data Loading
print("Step 1: Loading data from Excel files...")

# Load the primary data file, keeping all original columns.
df_primary = pd.read_excel(
    io=PRIMARY_FILE_PATH,
    sheet_name=PRIMARY_SHEET_NAME,
)
# Store the original column order to use for the final output
original_columns = df_primary.columns.tolist()
print(f"Primary file loaded successfully with {len(original_columns)} columns.")

# Load ONLY the required column from the reference file to save memory.
df_reference = pd.read_excel(
    io=REFERENCE_FILE_PATH,
    sheet_name=REFERENCE_SHEET_NAME,
    usecols=[REFERENCE_COLUMN_NAME]
)
# Drop any empty rows from the reference column
df_reference.dropna(subset=[REFERENCE_COLUMN_NAME], inplace=True)
print("Reference file loaded.")


# 4. Transformation Logic
print("Step 2: Processing and standardizing data...")

# --- Part A: Build the Canonical Mapping Dictionary ---
# Create a highly efficient lookup map (dictionary).
# The key is the standardized format (e.g., 'CityA|CityB').
# The value is the desired format from the reference file (e.g., 'CityA - CityB').
print("Building a reference map of correct connection names...")
canonical_map = {
    create_canonical_key(name): name 
    for name in df_reference[REFERENCE_COLUMN_NAME]
}

# --- Part B: Apply the Transformation to the Primary DataFrame ---
# Create the standardized key for the primary column.
df_primary['canonical_key'] = df_primary[PRIMARY_COLUMN_NAME].apply(create_canonical_key)

# Use the map to find the correct format.
# If a key exists in our map, use the map's value; otherwise, keep the original name.
# This preserves names that don't have a match in the reference file.
df_primary[PRIMARY_COLUMN_NAME] = df_primary['canonical_key'].map(canonical_map).fillna(df_primary[PRIMARY_COLUMN_NAME])

print("Data transformation complete. All other columns are preserved.")


# 5. Data Export
print("Step 3: Preparing final output and updating the original Excel file...")

# --- Part A: Assemble the Final DataFrame ---
# Select only the original columns in their original order.
# This step drops the temporary 'canonical_key' helper column.
df_final = df_primary[original_columns]

# --- Part B: Export to Original Excel File ---
# Use ExcelWriter in append mode with 'replace' to overwrite the specific sheet
# while leaving other sheets in the workbook untouched.
try:
    with pd.ExcelWriter(
        PRIMARY_FILE_PATH,
        mode="a",
        engine="openpyxl",
        if_sheet_exists="replace"
    ) as writer:
        df_final.to_excel(writer, sheet_name=PRIMARY_SHEET_NAME, index=False)
    
    print(f"✅ Success! Sheet '{PRIMARY_SHEET_NAME}' in '{PRIMARY_FILE_PATH}' has been updated.")

except FileNotFoundError:
    print(f"❌ Error: The file was not found at {PRIMARY_FILE_PATH}. Please check the path.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

Step 1: Loading data from Excel files...
Primary file loaded successfully with 23 columns.
Reference file loaded.
Step 2: Processing and standardizing data...
Building a reference map of correct connection names...
Data transformation complete. All other columns are preserved.
Step 3: Preparing final output and updating the original Excel file...
✅ Success! Sheet 'Filter_P' in 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx' has been updated.


In [ ]:
import pandas as pd
import os

# --- Configuration ---
# Please update these variables with your actual file names, sheet names, and column name.

# Source File: The file you are copying FROM.
source_file_path = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Output File.xlsx"
source_sheet_names = ['Filter_P', 'Filter_W']  # List of source sheets to process

# Destination File: The file you are copying TO. This file will be modified.
dest_file_path = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\WLP_NMS_services_UTL_comparsion_file_31_July_2025.xlsx"
dest_sheet_names = ['Only Protect', 'A end comparisonM&A']  # Corresponding destination sheets

# Key Column: The column name that uniquely identifies a row and is present in both files.
key_column_name = 'NE-SHELFSLOT-OPSSW'

# --- End of Configuration ---


def sync_excel_sheets(source_path, source_sheets, dest_path, dest_sheets, key_column):
    """
    Compares specified sheets from two Excel files based on a key column and copies missing rows
    from the source to the destination file. This function updates specified sheets while preserving
    all other sheets and existing rows in the destination file.
    """
    print("--- Ciena Excel Sheet Synchronizer (Multi-Sheet Safe) ---")

    # Validate input
    if len(source_sheets) != len(dest_sheets):
        print("Error: The number of source sheets must match the number of destination sheets.")
        return

    try:
        # Load the entire destination file to preserve all other sheets
        dest_file_data = pd.read_excel(dest_path, sheet_name=None)

        # Iterate through the specified sheets
        for source_sheet, dest_sheet in zip(source_sheets, dest_sheets):
            print(f"\nProcessing Source Sheet: '{source_sheet}' -> Destination Sheet: '{dest_sheet}'")

            # --- 1. Load Data from Excel Files ---
            try:
                df_source = pd.read_excel(source_path, sheet_name=source_sheet)
                df_dest = dest_file_data.get(dest_sheet, pd.DataFrame(columns=df_source.columns))
            except Exception as e:
                print(f"Error while reading sheets: {e}")
                return

            # --- 2. Validate Key Column ---
            if key_column not in df_source.columns:
                print(f"Error: Key column '{key_column}' not found in the source sheet '{source_sheet}'.")
                return
            if key_column not in df_dest.columns:
                print(f"Warning: Key column '{key_column}' not found in the destination sheet '{dest_sheet}'. Creating an empty sheet.")
                df_dest = pd.DataFrame(columns=df_source.columns)

            print(f"Using '{key_column}' as the key for comparison.")

            # --- 3. Identify Rows to be Added ---
            dest_keys = set(df_dest[key_column].unique())
            rows_to_add = df_source[~df_source[key_column].isin(dest_keys)]

            # --- 4. Append Data ---
            if rows_to_add.empty:
                print(f"\nResult: No new rows to add for sheet '{dest_sheet}'. The destination sheet is already up-to-date.")
            else:
                num_rows = len(rows_to_add)
                print(f"\nFound {num_rows} new row(s) to add to sheet '{dest_sheet}'.")

                df_dest = pd.concat([df_dest, rows_to_add], ignore_index=True)

            # Update the sheet in the destination file data
            dest_file_data[dest_sheet] = df_dest

        # --- 5. Save Updated Data ---
        try:
            print(f"\nSaving updated data to '{dest_path}'...")
            with pd.ExcelWriter(dest_path, engine='openpyxl') as writer:
                for sheet_name, data in dest_file_data.items():
                    data.to_excel(writer, sheet_name=sheet_name, index=False)
            print("Success! The destination file has been updated.")
        except Exception as e:
            print(f"An error occurred while saving the file: {e}")
            return

    except FileNotFoundError as e:
        print(f"Error: The file {e.filename} was not found. Please check the file path.")
        return
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return

    print("\n--- Synchronization Complete ---")


# --- Run the function ---
if __name__ == "__main__":
    sync_excel_sheets(
        source_file_path,
        source_sheet_names,
        dest_file_path,
        dest_sheet_names,
        key_column_name
    )

--- Ciena Excel Sheet Synchronizer (Multi-Sheet Safe) ---


c:\Users\mangoel\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\mangoel\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)



Processing Source Sheet: 'Filter_P' -> Destination Sheet: 'Only Protect'
Using 'NE-SHELFSLOT-OPSSW' as the key for comparison.

Found 2073 new row(s) to add to sheet 'Only Protect'.

Processing Source Sheet: 'Filter_W' -> Destination Sheet: 'A end comparisonM&A'


C:\Users\mangoel\AppData\Local\Temp\ipykernel_28336\3381045377.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_dest = pd.concat([df_dest, rows_to_add], ignore_index=True)


Using 'NE-SHELFSLOT-OPSSW' as the key for comparison.

Found 2073 new row(s) to add to sheet 'A end comparisonM&A'.

Saving updated data to 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\WLP_NMS_services_UTL_comparsion_file_31_July_2025.xlsx'...


C:\Users\mangoel\AppData\Local\Temp\ipykernel_28336\3381045377.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_dest = pd.concat([df_dest, rows_to_add], ignore_index=True)


Success! The destination file has been updated.

--- Synchronization Complete ---


In [ ]:
import os
import pandas as pd

def append_csv_files(folder_path, output_file):
    dataframes = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            print(f"Reading file: {file_path}")
            df = pd.read_csv(file_path)
            dataframes.append(df)

    combined_df = pd.concat(dataframes, ignore_index=True)

    combined_df.to_csv(output_file, index=False)
    print(f"Appended CSV saved to: {output_file}")

folder_path = r'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Utilization'
output_file = r'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Appended.csv'
append_csv_files(folder_path, output_file)

Reading file: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Utilization\performance-measurements (10).csv
Reading file: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Utilization\performance-measurements (11).csv
Reading file: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Utilization\performance-measurements (12).csv
Reading file: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Utilization\performance-measurements (13).csv
Reading file: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Utilization\performance-measurements (14).csv
Reading file: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Utilization\performance-measurements (7).csv
Reading file: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Utilization\performance-measurements (8).csv
Reading file: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Utilization\performance-

C:\Users\mangoel\AppData\Local\Temp\ipykernel_28336\335753297.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dataframes, ignore_index=True)


Appended CSV saved to: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Appended.csv


In [ ]:
import pandas as pd

def process_csv(file_path, start_column_index):
    """
    Process the input CSV file to add a 'reference' column and calculate the max numerical value
    between the specified starting column and the last column with values.

    Parameters:
        file_path (str): Path to the CSV file.
        start_column_index (int): The index of the starting column (e.g., column 'I').
    """
    # Read the CSV file into a DataFrame with low_memory=False
    df = pd.read_csv(file_path, low_memory=False)

    # Create the 'reference' column by combining values from specified columns
    df['reference'] = df[['Network element', 'Measurement point', 'Parameter', 'Direction']].apply(
        lambda row: '_'.join(row.values.astype(str)), axis=1
    )

    # Get the column names from the starting column index to the last column
    columns_with_values = df.iloc[:, start_column_index:].select_dtypes(include=['number']).columns

    # Calculate the maximum numerical value for each row between the specified columns
    df['Max'] = df[columns_with_values].max(axis=1)

    # Save the updated DataFrame back to the same CSV file
    df.to_csv(file_path, index=False)

    print(f"Updated CSV file saved: {file_path}")

# Example usage
file_path = r'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Appended.csv'  # Replace with the path to your CSV file
start_column_index = 8  # Replace with the column index for 'I' (e.g., 8 for the 9th column)
process_csv(file_path, start_column_index)

Updated CSV file saved: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Appended.csv


In [ ]:
import openpyxl
import pandas as pd

def update_excel_from_csv(target_file, source_csv, target_sheet_name, columns_to_copy):
    try:
        # Load the Target Excel File
        target_wb = openpyxl.load_workbook(target_file)
        
        # Check if the target sheet exists
        if target_sheet_name not in target_wb.sheetnames:
            print(f"Sheet '{target_sheet_name}' does not exist in the target file.")
            return
        
        # Load the Target Sheet
        target_sheet = target_wb[target_sheet_name]
        
        # Clear all data in the target sheet
        target_sheet.delete_rows(1, target_sheet.max_row)
        
        # Load the Source CSV File
        source_data = pd.read_csv(source_csv, low_memory=False)
        
        # Check if all required columns exist in the source CSV
        missing_columns = [col for col in columns_to_copy if col not in source_data.columns]
        if missing_columns:
            print(f"Missing columns in source CSV file: {missing_columns}")
            return
        
        # Write the headers to the target sheet
        for col_idx, column_name in enumerate(columns_to_copy, start=1):
            target_sheet.cell(row=1, column=col_idx, value=column_name)
        
        # Copy data from the source CSV to the target sheet
        for row_idx, row in source_data.iterrows():
            for col_idx, column_name in enumerate(columns_to_copy, start=1):
                target_sheet.cell(row=row_idx + 2, column=col_idx, value=row[column_name])
        
        # Save changes to the target file
        target_wb.save(target_file)
        print(f"Data successfully copied to sheet '{target_sheet_name}' in '{target_file}'.")
    
    except Exception as e:
        print(f"An error occurred: {e}")

# Define file paths and parameters
target_file = r'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\WLP_NMS_services_UTL_comparsion_file_31_July_2025.xlsx'
source_csv = r'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\Appended.csv'
target_sheet_name = "PMcombine-June2025"
columns_to_copy = ['Network element', 'Measurement point', 'Parameter', 'Direction', 'reference', 'Max']

# Call the function
update_excel_from_csv(target_file, source_csv, target_sheet_name, columns_to_copy)

Data successfully copied to sheet 'PMcombine-June2025' in 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\WLP_NMS_services_UTL_comparsion_file_31_July_2025.xlsx'.


In [ ]:
import pandas as pd
from openpyxl import Workbook, load_workbook

def add_new_sheet_from_csv(csv_file_path, excel_file_path, new_sheet_name, columns_to_copy):
    try:
        # Load the CSV file into a pandas DataFrame
        df = pd.read_csv(csv_file_path, low_memory=False)
        
        # Check if all required columns exist in the CSV file
        missing_columns = [col for col in columns_to_copy if col not in df.columns]
        if missing_columns:
            print(f"Missing columns in CSV file: {missing_columns}")
            return
        
        # Filter the DataFrame to include only the specified columns
        filtered_df = df[columns_to_copy]
        
        # Check if the Excel file exists
        try:
            wb = load_workbook(excel_file_path)
        except FileNotFoundError:
            # If the file doesn't exist, create a new workbook
            wb = Workbook()
            wb.remove(wb.active)  # Remove the default sheet
        
        # Create a new sheet in the Excel file
        if new_sheet_name in wb.sheetnames:
            print(f"Sheet '{new_sheet_name}' already exists. Overwriting it.")
            del wb[new_sheet_name]
        
        new_sheet = wb.create_sheet(title=new_sheet_name)
        
        # Write the DataFrame to the new sheet
        for col_idx, column_name in enumerate(columns_to_copy, start=1):
            new_sheet.cell(row=1, column=col_idx, value=column_name)  # Write headers
        
        for row_idx, row in filtered_df.iterrows():
            for col_idx, value in enumerate(row, start=1):
                new_sheet.cell(row=row_idx + 2, column=col_idx, value=value)  # Write data
        
        # Save the updated Excel file
        wb.save(excel_file_path)
        print(f"Data successfully copied to sheet '{new_sheet_name}' in '{excel_file_path}'.")
    
    except Exception as e:
        print(f"An error occurred: {e}")

# Define file paths and parameters
csv_file_path = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\OPTICAL_SERVICES_REPORT_V2.csv"  # Replace with the actual path to your CSV file
excel_file_path = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\OPTICAL_SERVICES_REPORT_V2.xlsx"  # Replace with the actual path to your Excel file
new_sheet_name = "new"
columns_to_copy = ['Name', 'Layer rate qualifier', 'Endpoint', 'Endpoint1']

# Call the function
add_new_sheet_from_csv(csv_file_path, excel_file_path, new_sheet_name, columns_to_copy)

Data successfully copied to sheet 'new' in 'C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\OPTICAL_SERVICES_REPORT_V2.xlsx'.


In [ ]:
import pandas as pd

def filter_excel(file_path):
    df = pd.read_excel(file_path)
    filtered_df = df[(df['Name'].notna()) & (df['Layer rate qualifier'] == '100GE')]
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
        filtered_df.to_excel(writer, index=False, sheet_name='Filtered Data')
    print("Done.")
file_path = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\OPTICAL_SERVICES_REPORT_V2.xlsx"
filter_excel(file_path)

Done.


In [ ]:
import pandas as pd

def stack_endpoints(input_file, output_file):
    # Load the Excel file into a Pandas DataFrame
    df = pd.read_excel(input_file)

    # Check if the required columns exist in the input file
    required_columns = ['Name', 'Layer rate qualifier', 'Endpoint', 'Endpoint1']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"The input file must contain the following columns: {required_columns}")

    # Replace missing values in 'Endpoint 1' and 'Endpoint 2' with a placeholder (e.g., 'N/A')
    df['Endpoint'] = df['Endpoint'].fillna('N/A')
    df['Endpoint1'] = df['Endpoint1'].fillna('N/A')

    # Create a new DataFrame by stacking 'Endpoint 1' and 'Endpoint 2'
    stacked_df = pd.DataFrame({
        'Name': pd.concat([df['Name'], df['Name']], ignore_index=True),
        'Layer rate qualifier': pd.concat([df['Layer rate qualifier'], df['Layer rate qualifier']], ignore_index=True),
        'Endpoint': pd.concat([df['Endpoint'], df['Endpoint1']], ignore_index=True)
    })

    # Save the new DataFrame to a new Excel file
    stacked_df.to_excel(output_file, index=False, sheet_name='Stacked Data')

    print(f"Successfully created the new Excel file: {output_file}")
    print(f"Original rows: {len(df)}, New rows: {len(stacked_df)}")

# Example usage
input_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\OPTICAL_SERVICES_REPORT_V2.xlsx"  # Replace with your input file path
output_file = r"C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\OPTICAL_SERVICES_REPORT_V2_.xlsx"  # Replace with your desired output file name
stack_endpoints(input_file, output_file)

Successfully created the new Excel file: C:\Users\mangoel\OneDrive - Ciena Corporation\WLP_automation_folder\OPTICAL_SERVICES_REPORT_V2_.xlsx
Original rows: 9364, New rows: 18728
